# Закон джунглей

## Логика

Изначально задается размер поля **SizeX, SizeY**. Количество существ каждого вида (хищник/препятствие/жертва), а также созаются два класса **Entity** и **Field**. В одном, соответсвенно, создается существо, в другом реализуются функции для поведения его на поле.

In [ ]:
class Entity:

    PLifeTime = 2

    VLifeTime = 9





    def __init__(self, x, y,lifetime):

        self.x = x

        self.y = y

        self.lifetime = lifetime;

        self.borntime = 3;

В зависимости от типа (хищник/жертва) существу присваивается время жизни (**PLifeTime/VLifeTime**), период для размножения **borntime** и координаты на поле.

Чтобы существа двигались не совсем рандомно, на каждом шаге каждой свободной клетке массива будем стваить в соответствие число, которое будет равно сумме шагов каждого хищника до данной клетки. Скажем, например, если хищнику нужно больше ходов чтобы добраться до определенной клетки, то жертве лучше идти в эту клетку. То же самое и для хищника.

В Field реализованы следующие функции:

* Инициализация (то же что и в примере)  


* update(self) реализует шаг на поле.  


* toField(self,items,pfield, let) отвечает за процессы рождения и смерти. Проверяет счетчики времени жизни и периода рождения



* eat(self, item) поедание жертв



* born(self,items, item, pfield,lifetime) рождение



* findtheway(self,item,pfield) поиск хода


* setProbabilityForOne(self, field, predX, predY) инициализация расстояний до данного хищника



* setProbability(self) суммирование этих расстояний для всех хищников




* inXY(self,x,y) проверка на пустоту

 Заполняем "поле вероятности" для одного хищника. Это просто сумма расстояний от него до данной клетки.

In [ ]:
def setProbabilityForOne(self, field, predX, predY):

        for x, y in self.xy:

            field[x][y] = field[x][y] + abs(x  - predX) + abs(y - predY)

        return field

 Далее суммируем эти значения по сем хищникам. При этом в клетки где находятся стены кладем отрицательное значение, чтобы потом их не рассматривать. Возвращаем заполненное поле.

In [ ]:
def setProbability(self):

        pfield = [[0] * SizeY for y in range(SizeX)]#сначала запихаем 0

        for predator in self.predators:

            pfield = self.setProbabilityForOne(pfield,predator.x,predator.y)#прогоняем каждого

        for walls in self.walls:

            pfield[walls.x][walls.y] = -1#в клетках со стенами -1

        return pfield

**findtheway(self,item,pfield)**, определяет куда каждому существу лучше сделать ход. Нужна клетка с максимальным значением. Если ход удовлетворяет параметрам поля и в новой клетке пусто, заполняем варианты хода вероятностью. Выбираем максимальный и идем туда. Если вдруг существо в углу окружено стенами и тд. то оно остается на месте.

In [ ]:
def findtheway(self,item,pfield):



        pway = [ -1 for x in range(4) ]

        way = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for w in way:

            x, y = w

            if item.x + x > 0 and item.x + x < SizeX and item.y + y > 0 and item.y + y < SizeY and not self.inXY(item.x + x,item.y + y):

                pway[way.index(w)] = pfield[item.x + x][item.y + y]

        p = max(pway)

        if p > 0 and p < SizeX+SizeY-1:

            return way[pway.index(p)]

        else:

            return (0, 0)

Функция **inXY** проверяет свободна ли клетка.

In [ ]:
def inXY(self,x,y):

        for v in self.victims:

            if v.x == x and v.y == y:

                return True

        for p in self.predators:

            if p.x == x and p.y == y:

                return True

        return False

После того как мы разобрались как существа перемещаются, надо разобраться как они рождаются, умирают и едят. 


За рождение и смерть отвечает функция **toField**

Величина время жизни lifetime зависит от того хищников или жертв обрабатываем (нужно для born). Далее для кажного:

- Если время рожать (borntime<=0), обновляем цикл рождения и рожаем.
- Иначе уменьшаем время до размножения.


- Если время жизни кончилось, то ничего не поделать, пора умирать
- Иначе приближаем время смерти :)

In [ ]:
def toField(self,items,pfield, let):

        if let == 'V':

            lifetime = Entity.VLifeTime

        else:

            lifetime = Entity.PLifeTime

        for item in items:

            if item.borntime  <= 0:

                item.borntime = 3

                self.born(items,item, pfield,lifetime)

            else:

                item.borntime -= 1

            if item.lifetime <= 0:

                items.remove(item)

                continue

            else:

                item.lifetime-=1

            self.field[item.x][item.y] = let

Соответственно сама функция рождения. Если соседняя клетка в поле, и там пусто, то добавляем новое существо туда.

In [ ]:
def born(self,items, item, pfield,lifetime):

        way = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for w in way:

            x, y = w

            if not self.inXY(item.x + x,item.y + y) and \

                item.x + x >= 0 and item.x + x < SizeX and item.y + y >= 0 and item.y + y < SizeY \

                and not pfield[item.x + x][item.x + x] == -1:

                items.append(Entity(item.x+x,item.y+y,lifetime))

                return

И последнее это ням-ням. Если жертва рядом с хищником, он откусывает от нее кусочек, увеличивает продолжительность своей жизни на 1, и приближает время размножения.

In [ ]:
def eat(self, item):

        for victim in self.victims:

            if abs(item.x - victim.x) <= 1 and abs(item.y - victim.y) <= 1:

                victim.lifetime -= 3

                item.lifetime += 1

                item.borntime -= 1

Функция update все соединяет:
    
* заполняем "поле вероятности"
* хищники ходят и "охотятся"
* хищнини toField -> умирают/размножаются


* ходят жертвы
* жертвы toField -> умирают/размножаются


In [ ]:
def update(self):

       #заполняем поле вероятности

        pfield = self.setProbability()

        self.field = [['*'] * SizeY for x in range(SizeX)]





        for predator in self.predators:

            x, y = self.findtheway(predator, pfield)

            predator.x += x

            predator.y += y

            self.eat(predator)

        self.toField(self.predators,pfield,'P')

        for victim in self.victims:

            x, y = self.findtheway(victim, pfield)

            victim.x += x

            victim.y += y



        self.toField(self.victims, pfield, 'V')

        for wall in self.walls:

            self.field[wall.x][wall.y] = 'W'

![alt-текст](https://pp.vk.me/c604423/v604423918/37b5a/pGcxJicdAyw.jpg "Необязательный титул")
![alt-текст](https://pp.vk.me/c604423/v604423918/37b9e/o5hwwN4XE0U.jpg "Необязательный титул")
![alt-текст](https://pp.vk.me/c604423/v604423918/37ba6/G4ZyFviVx5k.jpg "Необязательный титул")
![alt-текст](https://pp.vk.me/c604423/v604423918/37bae/DKlAp-5eDjE.jpg "Необязательный титул")
![alt-текст](https://pp.vk.me/c604423/v604423918/37bc6/Ef0nxe9-fi0.jpg "Необязательный титул")
![alt-текст](https://pp.vk.me/c604423/v604423918/37bb6/mODYrYor0FQ.jpg "Необязательный титул")
![alt-текст](https://pp.vk.me/c604423/v604423918/37bbe/LIbb7IQC2-M.jpg "Необязательный титул")